# WFS vs. NFC-HOA
## Frequency vs. Time Domain

fs446

In [ ]:
!date +%F

We use this jupyter notebook to illustrate the difference of **Wave Field Synthesis** (WFS), i.e. a spatially unlimited Sound Field Synthesis (SFS) approach to synthesis a broadband wavefront vs. so called **Near Field Compensated Higher Order Ambisonics** (NFC-HOA), i.e. modal bandlimited SFS.

We use the Python SFS Toolbox https://github.com/sfstoolbox/sfs-python, here tested with version 0.5.0 https://github.com/sfstoolbox/sfs-python/releases/tag/0.5.0

### Init Stuff

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from scipy.signal import unit_impulse, gausspulse 
import sfs

plt.rcParams['figure.figsize'] = 5, 5  # inch
plt.rcParams['axes.grid'] = True
plt.rcParams['savefig.dpi'] = 300

In [ ]:
def plot_fd(d, sssel, ss, ttlstr):
    p = sfs.fd.synthesize(d, sssel, ssd, ss, grid=grid)

    dBrange = 60    
    ls_lvl = np.abs(ssd.a * d * sssel) / np.max(np.abs(ssd.a * d * sssel))
    ls_lvl = 20*np.log10(ls_lvl)
    ls_lvl = 1/dBrange * np.clip(ls_lvl, -dBrange, 0) + 1
    
    #amplitude
    vmin = -2
    vmax = 2
    bounds = np.arange(vmin, vmax+0.1, 0.1)
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    plt.figure()
    sfs.plot2d.amplitude(p, grid, vmin=vmin, vmax=vmax, norm=norm, cmap='RdBu_r')
    sfs.plot2d.loudspeakers(ssd.x, ssd.n, ls_lvl, size=0.25)
    plt.title(ttlstr)
    plt.grid()
    #plt.savefig(ttlstr + '_lin_fd.png')
    
    #level
    vmin = -dBrange
    step = 6
    vmax = 6
    bounds = np.arange(vmin,vmax+step,step)
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    plt.figure()
    sfs.plot2d.level(p, grid, vmin=vmin, vmax=vmax, norm=norm, cmap='viridis')
    sfs.plot2d.loudspeakers(ssd.x, ssd.n, ls_lvl, size=0.25)
    plt.grid()    
    plt.title(ttlstr) 
    #plt.savefig(ttlstr + '_db_fd.png')

In [ ]:
def plot_td(d, sssel, ss, t=0, ttlstr=''):
    p = sfs.td.synthesize(d, sssel, ssd, ss, grid=grid, observation_time=t)
    
    # level
    vmin = -60
    step = 6
    vmax = 6
    bounds = np.arange(vmin,vmax+step,step)
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    plt.figure()
    sfs.plot2d.level(p, grid, vmin=vmin, vmax=vmax, norm=norm, cmap='magma_r')
    sfs.plot2d.loudspeakers(ssd.x, ssd.n, sssel, size=0.25)
    plt.title(ttlstr)
    plt.grid()     
    #plt.savefig(ttlstr + '_db_td.png')

In [ ]:
R = 2  # m, secondary source distribution radius
N = 2**5  # number of secondary sources
ssd = sfs.array.circular(N, R)
grid = sfs.util.xyz_grid([-2.1, 2.1], [-2.1, 2.1],
                         np.mean(ssd.x[:,2]), spacing=0.01)
xref = [0, 0, 0]  # referencing point or WFS

## Frequency Domain / Monochromatic SFS

In [ ]:
f = sfs.default.c*1  # Hz
lmb = sfs.default.c/f  # m
print('frequency', f, 'Hz, wavelength', lmb, 'm')
omega = 2 * np.pi * f  # rad/s

### Point Source Characteristics

In [ ]:
klmb = 3
xs = [-klmb*lmb*np.cos(np.pi/4), klmb*lmb*np.sin(np.pi/4), 0]
#xs = [-2.8284271247461903, 2.82842712474619, 0]
As = np.linalg.norm(xs)*4*np.pi
rs = np.linalg.norm(xs)
ts = rs / sfs.default.c

### Plane Wave Characteristics

In [ ]:
pw_angle = 315  # propagating direction within xy-plane in deg
npw = sfs.util.direction_vector(np.radians(pw_angle), np.radians(90))

### SFS of Point Source 

In [ ]:
d, sssel, ssfunc = sfs.fd.wfs.point_25d(omega, ssd.x, ssd.n, xs)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
normalize = 1
plot_fd(As*d*normalize, tapering_window, ssfunc, 'Point Source WFS')

In [ ]:
d, sssel, ssfunc = sfs.fd.nfchoa.point_25d(omega, ssd.x, R, xs)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
plot_fd(As*d, tapering_window, ssfunc, 'Point Source NFC-HOA')

### SFS of Plane Wave 

In [ ]:
d, sssel, ssfunc = sfs.fd.wfs.plane_25d(omega, ssd.x, ssd.n, npw, xref=xref)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
plot_fd(d, tapering_window, ssfunc, 'Plane Wave WFS')

In [ ]:
d, sssel, ssfunc = sfs.fd.nfchoa.plane_25d(omega, ssd.x, R, npw)        
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
plot_fd(d, tapering_window, ssfunc, 'Plane Wave NFC-HOA')

## Time Domain  / Monochromatic SFS

In [ ]:
fs = 48000  # Hz, samling frequency
#dirac = unit_impulse(512), fs
dirac = gausspulse(np.arange(0,512)/fs, fc=f, bw=f/10, bwr=-6, tpr=-60, retquad=False, retenv=False), fs

### SFS of Point Source 

In [ ]:
delay, amplitude, sssel, ss = sfs.td.wfs.point_25d(ssd.x, ssd.n, xs)
weird_normalize = 2.5
d = sfs.td.wfs.driving_signals(delay, weird_normalize*amplitude, dirac)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
plot_td(d, tapering_window, ss, t=(ts+0/1000), ttlstr='Point Source WFS')

In [ ]:
delay, amplitude, sos, phaseshift, sssel, ss = sfs.td.nfchoa.point_25d(ssd.x, R, xs, fs)
d = sfs.td.nfchoa.driving_signals_25d(delay, amplitude, sos, phaseshift, dirac)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
plot_td(d, tapering_window, ss, t=(ts+0/1000), ttlstr='Point Source NFC-HOA')

### SFS of Plane Wave 

In [ ]:
delay, amplitude, sssel, ss = sfs.td.wfs.plane_25d(ssd.x, ssd.n, npw)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
d = sfs.td.wfs.driving_signals(delay, amplitude, dirac)
plot_td(d, tapering_window, ss, t=0/1000, ttlstr='Plane Wave WFS')

In [ ]:
delay, amplitude, sos, phaseshift, sssel, ss = sfs.td.nfchoa.plane_25d(ssd.x, R, npw, fs)
tapering_window = sfs.tapering.kaiser(sssel, beta=0)
d = sfs.td.nfchoa.driving_signals_25d(delay, amplitude, sos, phaseshift, dirac)
plot_td(d, tapering_window, ss, t=0/1000, ttlstr='Plane Wave NFC-HOA')